In [27]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from nolearn.dbn import DBN
from sklearn.metrics import classification_report, accuracy_score



In [28]:
# read training data from CSV file 
data = pd.read_csv('./dataset/train.csv')
print('data({0[0]},{0[1]})'.format(data.shape))

labels_flat = data[[0]].values.ravel()
print('labels_flat({0})'.format(len(labels_flat)))

images = data.iloc[:,1:].values
images = images.astype(np.float)
print('images({0[0]},{0[1]})'.format(images.shape))


data(42000,785)
labels_flat(42000)
images(42000,784)


In [29]:
# read test data from CSV file 
test_images = pd.read_csv('./dataset/test.csv').values
test_images = test_images.astype(np.float)

print('test_images({0[0]},{0[1]})'.format(test_images.shape))


test_images(28000,784)


In [30]:

def train_test_prep(data):
    '''
    This function will load the MNIST data, scale it to a 0 to 1 range, and split it into test/train sets. 
    '''
    
    image_data = data#fetch_mldata('MNIST Original') # Get the MNIST dataset.
    
    basic_x = data.iloc[:,1:].values#image_data.data
    basic_y = data[[0]].values.ravel()#image_data.target # Separate images from their final classification. 
    
    min_max_scaler = MinMaxScaler() # Create the MinMax object.
    basic_x = min_max_scaler.fit_transform(basic_x.astype(float)) # Scale pixel intensities only.
    
    x_train, x_test, y_train, y_test = train_test_split(basic_x, basic_y, 
                            test_size = 0.2, random_state = 0) # Split training/test.
    return x_train, x_test, y_train, y_test


In [41]:
def minMaxScaler(test_images):
    min_max_scaler = MinMaxScaler() # Create the MinMax object.
    return min_max_scaler.fit_transform(test_images) # Scale pixel intensities only.
test_images = minMaxScaler(test_images)

In [31]:
x_train, x_test, y_train, y_test = train_test_prep(data)

In [35]:
print x_train.shape
print x_train[1].shape

print y_train.shape

(33600L, 784L)
(784L,)
(33600L,)


In [46]:
    dbn_model = DBN([x_train.shape[1], 300, 10],
                    learn_rates = 0.3,
                    learn_rate_decays = 0.9,
                    epochs = 10,
                    dropouts = 0.25, # Express the percentage of nodes that will be randomly dropped as a decimal.
                    verbose = 1)

In [47]:
dbn_model.fit(x_train, y_train)

[DBN] fitting X.shape=(33600L, 784L)
[DBN] layers [784L, 300, 10]
[DBN] Fine-tune...


100%


Epoch 1:
  loss 0.438534259668
  err  0.136160714286
  (0:00:09)


100%


Epoch 2:
  loss 0.283104429564
  err  0.0850297619048
  (0:00:09)


...

Epoch 8:
  loss 0.111093119758
  err  0.0360416666667
  (0:00:09)


100%


Epoch 9:
  loss 0.108087110074
  err  0.035625
  (0:00:09)


100%


Epoch 10:
  loss 0.0945011895363
  err  0.0303571428571
  (0:00:09)


In [48]:
y_true, y_pred = y_test, dbn_model.predict(x_test) # Get our predictions
print(classification_report(y_true, y_pred)) # Classification on each digit

             precision    recall  f1-score   support

          0       0.99      0.99      0.99       813
          1       0.98      0.99      0.98       961
          2       0.96      0.98      0.97       860
          3       0.99      0.94      0.97       863
          4       0.99      0.96      0.97       827
          5       0.97      0.98      0.98       756
          6       0.97      1.00      0.98       841
          7       0.98      0.97      0.97       899
          8       0.95      0.98      0.96       768
          9       0.96      0.96      0.96       812

avg / total       0.97      0.97      0.97      8400



In [49]:
def saveFileForSubmission(predicted_lables,custonFileName='submission.csv',customHeader='ImageId,Label'):
    result = np.c_[range(1,len(predicted_lables)+1),predicted_lables]

    np.savetxt(custonFileName, 
           result.astype(int), 
           delimiter=',', 
           header = customHeader, 
           comments = '', 
           fmt='%u,%u')

In [50]:
y_pred = dbn_model.predict(test_images) # Get our predictions


In [51]:
saveFileForSubmission(y_pred,'submission3DBN.csv')